In [1]:
import os

os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["CUDA_LAUNCH_BLOCKING"] = '1'
os.environ['WANDB_DISABLED'] = 'True'

In [2]:
import torch
import re
import random
import extendNER_new_bioe as ex #bio or bioe
from transformers import ElectraForTokenClassification, AutoConfig, AutoTokenizer
from transformers import AdamW
import pandas as pd
from torch.utils.data import Dataset as DS
from datasets import Dataset
from transformers import DefaultDataCollator
from transformers import Trainer, TrainingArguments
from seqeval.metrics import classification_report
from sklearn.metrics import f1_score
import numpy as np
from datasets import load_metric
import wandb

/home/jjs970612/anaconda3/envs/temp/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from seqeval.scheme import IOB1, IOB2, IOE1, IOE2, IOBES, BILOU, Entities, Prefix, Tag
from seqeval.scheme import IOBES

In [4]:
permutations = [['ORG', 'PER', 'CVL', 'DAT', 'LOC', 'QNT'],
               ['DAT', 'QNT', 'PER', 'LOC', 'ORG', 'CVL'],
               ['CVL', 'LOC', 'ORG', 'QNT', 'DAT', 'PER'],
               ['QNT', 'ORG', 'DAT', 'PER', 'CVL', 'LOC'],
               ['LOC', 'CVL', 'QNT', 'ORG', 'PER', 'DAT'],
               ['PER', 'DAT', 'LOC', 'CVL', 'QNT', 'ORG']]

In [5]:
class NERDataset(DS):
    def __init__(self, dataset, tokenizer, label2id, max_length):
        super().__init__()
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length
        self.dataset = dataset
        self.data = []
        
        for i in range(len(self.dataset['label'])):
            text = dataset['label'][i]
            tagged_words = re.findall('<.*?:.*?>', text)
            
            word2ids = dict()
            for tagged_word in tagged_words:
                tag_splited = tagged_word.strip('<>').split(':')
                tag = tag_splited[-1]
                word = ':'.join(tag_splited[:-1])
                    
                word_tok = self.tokenizer.encode(word)[1:-1]
                if word not in word2ids:

                    label_id = [self.label2id['B-'+tag]]
                    if len(word_tok) > 1 :
                        label_id.extend([self.label2id['I-'+tag]] * (len(word_tok)-1))
                    word2ids[word] = {
                        'target_ids': word_tok,
                        'label_id': label_id 
                    }
                    text = text.replace(tagged_word, word)

            tokenized = self.tokenizer(text, truncation=True, max_length=self.max_length, padding='max_length')
            if 0 in tokenized['input_ids']:
                tok_length = tokenized['input_ids'].index(0)
            else:
                tok_length = self.max_length
            label_input = tokenized['input_ids'][:tok_length]
            labels = self._gen_labels(label_input, word2ids)
            labels.insert(0, -100)
            pad = [-100] * (self.max_length - len(labels))
            labels.extend(pad)

            temp = {
                'input_ids' : tokenized['input_ids'],
                'attention_mask' : tokenized['attention_mask'],
                'labels' : labels
            }
            
            self.data.append(temp)
        
    def _gen_labels(self, input_ids, word2ids):
        sequence = input_ids[1:-1]
        labels = [0] * len(sequence)
        
        for v in word2ids.values():
            target_ids = v['target_ids']
            label_id = v['label_id']
            
            i=0
            target_ids_length = len(target_ids)
            
            while i < len(sequence):
                if sequence[i:i + target_ids_length] == target_ids:
                    labels[i:i + target_ids_length] = label_id
                    i = i + target_ids_length
                else:
                    i += 1
                    
        return labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.data[idx]['input_ids'],
            'attention_mask': self.data[idx]['attention_mask'],
            'labels': self.data[idx]['labels'],
        }

In [6]:
class IOBE(IOBES):
    allowed_prefix = Prefix.I | Prefix.O | Prefix.B | Prefix.E
    start_patterns = {
        (Prefix.ANY, Prefix.B, Tag.ANY),
        (Prefix.ANY, Prefix.S, Tag.ANY)
    }
    inside_patterns = {
        (Prefix.B, Prefix.I, Tag.SAME),
        (Prefix.B, Prefix.E, Tag.SAME),
        (Prefix.I, Prefix.I, Tag.SAME),
        (Prefix.I, Prefix.E, Tag.SAME)
    }
    end_patterns = {
        (Prefix.S, Prefix.ANY, Tag.ANY),
        (Prefix.E, Prefix.ANY, Tag.ANY),
        (Prefix.B, Prefix.O, Tag.ANY),
        (Prefix.B, Prefix.I, Tag.DIFF),
        (Prefix.B, Prefix.B, Tag.ANY),
    }

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    #import pdb;pdb.set_trace()
    predictions = predictions.flatten()
    labels = labels.flatten()
    npre = []
    nlab = []
 
    for i in range(len(labels)):
        if labels[i] != -100:
            npre.append(predictions[i])
            nlab.append(labels[i])
    npre = torch.tensor(npre)
    nlab = torch.tensor(nlab)
    
    label_indices = label_arr.copy()
    npre = [label_indices[pred] for pred in npre]
    nlab = [label_indices[label] for label in nlab]
    del label_indices[label_indices.index("O")]
    entity_level_metrics = classification_report(
        [nlab], [npre], digits=3,
        suffix=False,
        mode= 'strict', scheme=IOBE, 
        zero_division=True, output_dict=True
    )

    metrics = {}
    
    for key in entity_level_metrics.keys():
        if len(key) == 3:
            metrics[key+"_f1"] = entity_level_metrics[key]['f1-score']
            metrics[key+"_recall"] = entity_level_metrics[key]['recall']
            metrics[key+"_precision"] = entity_level_metrics[key]['precision']
            
        if key == 'macro avg':
            metrics["entity_macro_f1"] = entity_level_metrics['macro avg']['f1-score']
            metrics["entity_macro_precision"] = entity_level_metrics['macro avg']['precision']
            metrics["entity_macro_recall"] = entity_level_metrics['macro avg']['recall']
            
    return metrics

In [8]:
def cl_train(config):
    
    teacher = ElectraForTokenClassification.from_pretrained(config['teacher_dir'])
    student = ElectraForTokenClassification.from_pretrained(config['base_model_dir'], num_labels=(config['step']*2+1))
    
    ent = config['new_entity']
    label2id = teacher.config.label2id
    label2id['B-'+ent] = student.config.num_labels-2
    label2id['I-'+ent] = student.config.num_labels-1
    id2label = {label2id[label] : label for label in label2id.keys()}
    
    student.config.label2id = label2id
    student.config.id2label = id2label

    sorted_dict = dict(sorted(id2label.items()))
    
    global label_arr
    
    label_arr = []
    for v in sorted_dict.values():
        label_arr.append(v)
    
    tokenizer = AutoTokenizer.from_pretrained(config['teacher_dir'])
    
    train_file = pd.read_csv(config['train_file'])
    valid_file = pd.read_csv(config['valid_file'], sep='\t')
    
    train_dataset = NERDataset(train_file, tokenizer=tokenizer, max_length=300, label2id=label2id)
    valid_dataset = NERDataset(valid_file, tokenizer=tokenizer, max_length=300, label2id=label2id)
    
    temp_train = []
    temp_valid = []
    for e in train_dataset:
        temp_train.append(e)
        
    for e in valid_dataset:
        temp_valid.append(e)   
        
    teacher_input = pd.DataFrame(temp_train)
    student_input = teacher_input.copy()
    teacher_input = teacher_input.drop(columns=['labels'])
    teacher_input = Dataset.from_pandas(teacher_input)
    
    data_collator = DefaultDataCollator()
    
    trainer = Trainer(
        model=teacher,
        data_collator=data_collator
    )

    prediction = trainer.predict(teacher_input)
    pred = np.argmax(prediction[0], axis=-1)
    pred = pred.tolist()
    
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            if student_input['labels'][i][j] == student.config.num_labels-1 or student_input['labels'][i][j] == student.config.num_labels-2:
                pred[i][j] = student_input['labels'][i][j]
    
    student_input['labels'] = pred

    valid_input = pd.DataFrame(temp_valid)

    student_input = Dataset.from_pandas(student_input)
    valid_input = Dataset.from_pandas(valid_input)
    
    data_collator = DefaultDataCollator()
    
    device = torch.device("cuda")
    student.to(device)
    
    training_args = TrainingArguments(
        output_dir=config['output_dir'],
        do_eval = True,
        learning_rate=config['learning_rate'],
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=config['train_epoch'],
        weight_decay=0.1,
        save_strategy = 'epoch',
        logging_strategy = 'epoch',
        evaluation_strategy = 'epoch',
        load_best_model_at_end = True,
        label_names = ['labels'],
        metric_for_best_model = 'entity_macro_f1',
        warmup_ratio = 0.05,
        no_cuda = False,
    )

    trainer = Trainer(
        model=student,
        args=training_args,
        train_dataset=student_input,
        eval_dataset = valid_input,
        data_collator=data_collator,
        tokenizer = tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.save_model(config['output_dir'] + '/final')
    
    for f_name in os.listdir(config['output_dir']):
        if f_name.startswith('checkpoint'):
            for f in os.listdir(config['output_dir']+'/'+f_name):
                os.remove(config['output_dir']+'/'+f_name+'/'+f)
            os.rmdir(config['output_dir']+'/'+f_name)
    

In [9]:
def test(config):
    
    model = ElectraForTokenClassification.from_pretrained(config['model'])
    
    global label_arr
    
    label_arr = []
    for v in model.config.id2label.values():
        label_arr.append(v)
    
    tokenizer = AutoTokenizer.from_pretrained(config['model'])
    
    test_file = pd.read_csv(config['test_file'], sep='\t')
    label2id = model.config.label2id
    
    test_dataset = NERDataset(test_file, tokenizer=tokenizer, max_length=300, label2id=label2id)
    
    device = torch.device("cuda")
    model.to(device)
    
    data_collator = DefaultDataCollator()
    
    training_args = TrainingArguments(
        output_dir=config['model'],
        per_device_eval_batch_size=32,
    )

    trainer = Trainer(
        args=training_args,
        model=model,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )
    
    metrics = trainer.evaluate(test_dataset)
    trainer.save_metrics(split='test', metrics=metrics)
    

In [10]:
import os

if(__name__=="__main__"):
    
    for per_num_int in range(1,7):
        per_num = str(per_num_int)
        permutation_dir = "train_data/perm_" + per_num

        data_file_dict = {
            "train" : [],
            "valid" : []
        }

        permutations = [['ORG', 'PER', 'CVL', 'DAT', 'LOC', 'QNT'],
                       ['DAT', 'QNT', 'PER', 'LOC', 'ORG', 'CVL'],
                       ['CVL', 'LOC', 'ORG', 'QNT', 'DAT', 'PER'],
                       ['QNT', 'ORG', 'DAT', 'PER', 'CVL', 'LOC'],
                       ['LOC', 'CVL', 'QNT', 'ORG', 'PER', 'DAT'],
                       ['PER', 'DAT', 'LOC', 'CVL', 'QNT', 'ORG']]

        for i in range(6):
            for f_name in os.listdir(permutation_dir + '/'):
                if f_name.startswith('d'+str(i)):
                    data_file_dict['train'].append(f_name)
                    break

            for f_name in os.listdir('test_data/perm' + str(per_num)):
                if f_name.startswith('eval_'+str(i+1)):
                    data_file_dict['valid'].append(f_name)
                    break

        for i in range(1, 6):

            new_ent = permutations[per_num_int-1][i]

            cl_config = {
                'base_model_dir' : 'monologg/koelectra-base-v3-discriminator',
                'teacher_dir' : 'self_train_result/perm_' + per_num + '/step' + str(i) + '/final',
                'new_entity' : new_ent,
                'train_file' : permutation_dir + '/' + data_file_dict['train'][i],
                'valid_file' : 'test_data/perm' + str(per_num) + '/' + data_file_dict['valid'][i],
                'output_dir' : 'self_train_result/perm_' + per_num + '/step' + str(i+1) + '/',
                'train_epoch' : 10,
                'learning_rate' : 5e-05,
                'step' : i+1
            }

            cl_train(cl_config)

            test_config = {
                'model':'self_train_result/perm_' + per_num + '/step' + str(i+1) + '/final',
                'test_file': 'test_data/perm' + str(per_num) + '/' + data_file_dict['valid'][i]
            }    

            test(test_config)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/jjs970612/anaconda3/envs/temp/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Org F1,Org Recall,Org Precision,Per F1,Per Recall,Per Precision,Entity Macro F1,Entity Macro Precision,Entity Macro Recall
1,0.291000,0.144272,0.264159,0.812977,0.157700,0.024242,0.012539,0.363636,0.144201,0.260668,0.412758
2,0.017500,0.066758,0.809936,0.891858,0.741799,0.913713,0.929467,0.898485,0.861825,0.820142,0.910662
3,0.008700,0.065827,0.840244,0.876590,0.806792,0.929293,0.937304,0.921418,0.884768,0.864105,0.906947
4,0.006300,0.075403,0.801743,0.936387,0.700952,0.921314,0.945141,0.898659,0.861528,0.799806,0.940764
5,0.005000,0.067578,0.825415,0.917303,0.750260,0.929512,0.940439,0.918836,0.877464,0.834548,0.928871
6,0.003900,0.077035,0.827709,0.889313,0.774086,0.930739,0.937304,0.924266,0.879224,0.849176,0.913309
7,0.003300,0.073221,0.825888,0.917303,0.751042,0.935937,0.938871,0.933022,0.880913,0.842032,0.928087
8,0.002900,0.079008,0.822654,0.914758,0.747401,0.935106,0.937304,0.932917,0.878880,0.840159,0.926031
9,0.002300,0.078604,0.814731,0.914758,0.734423,0.935004,0.935737,0.934272,0.874867,0.834348,0.925247
10,0.002100,0.081646,0.814521,0.913486,0.734903,0.936669,0.938871,0.934477,0.875595,0.834690,0.926179


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier

...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/jjs970612/anaconda3/envs/temp/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Cvl F1,Cvl Recall,Cvl Precision,Dat F1,Dat Recall,Dat Precision,Loc F1,Loc Recall,Loc Precision,Org F1,Org Recall,Org Precision,Per F1,Per Recall,Per Precision,Qnt F1,Qnt Recall,Qnt Precision,Entity Macro F1,Entity Macro Precision,Entity Macro Recall
1,0.478700,0.398382,0.724089,0.744853,0.704451,0.687161,0.581040,0.840708,0.278922,0.171540,0.745763,0.449612,0.854033,0.305124,0.810765,0.755486,0.874773,0.354260,0.537415,0.264214,0.550802,0.622505,0.607394
2,0.038000,0.200422,0.846367,0.899265,0.799346,0.795732,0.798165,0.793313,0.811631,0.938596,0.714922,0.780603,0.845070,0.725275,0.911901,0.924765,0.899390,0.677966,0.816327,0.579710,0.804033,0.751993,0.870365
3,0.016400,0.214214,0.820915,0.923529,0.738824,0.780924,0.801223,0.761628,0.850896,0.948343,0.771610,0.785512,0.888604,0.703854,0.917557,0.942006,0.894345,0.700855,0.836735,0.602941,0.809443,0.745534,0.890074
4,0.010600,0.230908,0.820295,0.921324,0.739233,0.794562,0.804281,0.785075,0.860915,0.889864,0.833790,0.790466,0.912932,0.696970,0.909506,0.937304,0.883309,0.731707,0.816327,0.662983,0.817908,0.766893,0.880339
5,0.007900,0.222081,0.863224,0.909559,0.821381,0.809231,0.804281,0.814241,0.865728,0.898635,0.835145,0.838631,0.878361,0.802339,0.909368,0.935737,0.884444,0.746082,0.809524,0.691860,0.838710,0.808235,0.872683
6,0.005900,0.239263,0.838495,0.925735,0.766281,0.804281,0.804281,0.804281,0.861150,0.897661,0.827493,0.827912,0.896287,0.769231,0.917305,0.938871,0.896707,0.759494,0.816327,0.710059,0.834773,0.795675,0.879860
7,0.005000,0.246720,0.841929,0.924265,0.773063,0.803053,0.804281,0.801829,0.858341,0.897661,0.822321,0.827219,0.895006,0.768977,0.916730,0.940439,0.894188,0.720238,0.823129,0.640212,0.827918,0.783432,0.880797
8,0.004200,0.251770,0.841965,0.926471,0.771586,0.812121,0.819572,0.804805,0.861409,0.899610,0.826321,0.831340,0.889885,0.780022,0.918836,0.940439,0.898204,0.737805,0.823129,0.668508,0.833913,0.791574,0.883184
9,0.003700,0.255656,0.835208,0.927941,0.759326,0.804249,0.810398,0.798193,0.859972,0.888889,0.832877,0.823045,0.896287,0.760870,0.911988,0.942006,0.883824,0.758621,0.823129,0.703488,0.832180,0.789763,0.881442
10,0.003400,0.258436,0.841472,0.925000,0.771779,0.804249,0.810398,0.798193,0.856740,0.888889,0.826836,0.828319,0.898848,0.768053,0.915332,0.940439,0.891530,0.758621,0.823129,0.703488,0.834122,0.793313,0.881117


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
